In [1]:
import numpy as np
import os
from pylatex import Document, Section, Subsection, Command, Tabular
from pylatex.utils import italic, NoEscape
from at2019dsg.data import bran_disc, t_peak_mjd, photometry_data, radio_data, radio_qs_epochs, xray_data, radio_rec, data_dir
import astropy.io.ascii

In [2]:
def fill_document(doc, df, headings):
    """Add a section, a subsection and some text to the document.

    :param doc: the document
    :type doc: :class:`pylatex.document.Document` instance
    """
    with doc.create(Section('A section')):
        doc.append('Some regular text and some ')
        doc.append(italic('italic text. '))

        with doc.create(Subsection('A subsection')):
            doc.append('Also some crazy characters: $&#{}')
            
            with doc.create(Subsection('Table of something')):
                with doc.create(Tabular('|' + "".join(["c|" for _ in headings]))) as table:
                    table.add_hline()
                    table.add_row(tuple([NoEscape(r"\textbf{" + x + "}") for x in headings]))
                    table.add_hline()
                    for _, row in df.iterrows():
                        entry = tuple([x for x in row])
                        table.add_row(entry)
                    table.add_hline()

In [3]:
offset = t_peak_mjd.mjd - bran_disc.mjd
photometry_data["#day_since_peak"] += offset
photometry_data["flux_Jy"] *= 1.e3
photometry_data["nu_rest"] /= 1.e14

flux_err = photometry_data["flux_Jy"]  * photometry_data["err_lum"]/photometry_data["lum"]

print(flux_err)

instruments = np.array(["Swift-UVOT" for _ in photometry_data["nu_rest"]])

mask = np.array(["." in x for x in photometry_data["band"]])

bands = np.array([x.split(".")[0] for x in photometry_data["band"][mask]])
tels = np.array([x.split(".")[1] for x in photometry_data["band"][mask]])
instruments[mask] = tels

instruments[instruments == "IOO"] = "LT (IOO)"

photometry_data["band"][mask] = bands

photometry_data = photometry_data.assign(instrument=instruments, flux_err=flux_err)


photometry_data = photometry_data[photometry_data["flux_Jy"] > 0.0]

photometry_data["#day_since_peak"] -= min(photometry_data["#day_since_peak"])
photometry_data = photometry_data.round(2)

photometry_data = photometry_data.sort_values(["nu_rest", "#day_since_peak"], ascending=True)

fields = ["#day_since_peak", "band", "flux_Jy", "flux_err", "nu_rest", "instrument"]

col_headings = [
    r"$\Delta_{t}$",
    r"Band",
    r"Flux [mJy]",
    r"Flux Error [mJy]", 
    r"$\nu [10^{14} Hz]]$",
    "Instrument"
]


# if __name__ == '__main__':
# Basic document
doc = Document('basic')

# fill_document(doc, photometry_data.iloc[:, : 4], col_headings)
fill_document(doc, photometry_data[fields], col_headings)


# Add stuff to the document
with doc.create(Section('A second section')):
    doc.append('Some text.')

tex = doc.dumps()  # The document as string in LaTeX syntax
print(tex)

0      0.014420
1      0.026061
2      0.019625
3      0.019090
4      0.019267
         ...   
263    0.004613
264    0.017771
265    0.017543
266    0.015838
267    0.012059
Length: 268, dtype: float64
\documentclass{article}%
\usepackage[T1]{fontenc}%
\usepackage[utf8]{inputenc}%
\usepackage{lmodern}%
\usepackage{textcomp}%
\usepackage{lastpage}%
%
%
%
\begin{document}%
\normalsize%
\section{A section}%
\label{sec:Asection}%
Some regular text and some %
\textit{italic text. }%
\subsection{A subsection}%
\label{subsec:Asubsection}%
Also some crazy characters: \$\&\#\{\}%
\subsection{Table of something}%
\label{subsec:Tableofsomething}%
\begin{tabular}{|c|c|c|c|c|c|}%
\hline%
\textbf{$\Delta_{t}$}&\textbf{Band}&\textbf{Flux [mJy]}&\textbf{Flux Error [mJy]}&\textbf{$\nu [10^{14} Hz]]$}&\textbf{Instrument}\\%
\hline%
34.02&i&0.19&0.02&4.23&LT (IOO)\\%
38.76&i&0.19&0.02&4.23&LT (IOO)\\%
47.32&i&0.17&0.02&4.23&LT (IOO)\\%
67.26&i&0.13&0.01&4.23&LT (IOO)\\%
0.0&r&0.09&0.01&4.96&ZTF\\%
10.4

<ipython-input-3-29559c26b8cf>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  photometry_data["band"][mask] = bands


In [4]:
radio_data["mjd"] -= bran_disc.mjd
radio_data["mjd"] = np.array([f"{x:.0f}" for x in radio_data["mjd"]])
radio_data = radio_data.assign(qs=radio_qs_epochs)

mask = radio_data["instrument"] != "MeerKAT"

flux_err = np.array(radio_data["flux_err"])

flux_err[mask] = (np.array(radio_data[mask]["flux_err"], dtype=np.float)**2. + (0.05*radio_data[mask]["flux"])**2.)**0.5


print(radio_data[mask]["flux_err"])
print(0.05*radio_data[mask]["flux"])

radio_data = radio_data.assign(qs=radio_qs_epochs, flux_err=flux_err)

print(radio_data[mask]["flux_err"])

# print(radio_data[mask]["flux_err"], radio_data[mask]["flux"], 0.05*radio_data[mask]["flux"])
# print(np.array(radio_data[mask]["flux_err"], dtype=np.float) + 0.05*radio_data[mask]["flux"])

# print(radio_data[mask]["flux_err"])

radio_data = radio_data.round(3)

radio_data = radio_data.sort_values(["qs", "frequency"], ascending=True)

fields = ["mjd", "frequency", "flux", "flux_err", "instrument"]

col_headings = [
    r"$\Delta_{t}$",
    r"$\nu$ [GHz]",
    r"Flux [mJy]",
    r"Flux Err [mJy]",
    "Instrument"
]


# if __name__ == '__main__':
# Basic document
doc = Document('basic')

# fill_document(doc, photometry_data.iloc[:, : 4], col_headings)
fill_document(doc, radio_data[fields], col_headings)


# Add stuff to the document
with doc.create(Section('A second section')):
    doc.append('Some text.')

tex = doc.dumps()  # The document as string in LaTeX syntax
print(tex)

0     0.028000
1     0.029000
2     0.028000
3     0.031000
4     0.028000
5     0.037000
6     0.029000
7     0.030000
8     0.030000
9     0.030000
10    0.029000
11    0.030000
12    0.034000
13    0.064000
14    0.050000
15    0.047000
16    0.037000
17    0.030000
18    0.033000
19    0.030000
20    0.030000
21    0.031000
22    0.030000
23    0.032000
24    0.032000
25    0.052000
26    0.039000
27    0.034000
28    0.036000
29    0.029000
30    0.030000
31    0.030000
32    0.028000
33    0.030000
34    0.028000
35    0.029000
36    0.030000
37    0.038720
38    0.066845
39    0.056853
40    0.043891
41    0.039147
Name: flux_err, dtype: float64
0     0.01450
1     0.02040
2     0.02200
3     0.02060
4     0.00515
5     0.01595
6     0.01620
7     0.02215
8     0.02790
9     0.03400
10    0.03650
11    0.03780
12    0.03855
13    0.01245
14    0.01725
15    0.01275
16    0.02095
17    0.03490
18    0.04145
19    0.04935
20    0.05585
21    0.05970
22    0.06190
23    0.06550
24 

In [5]:
xray_data["MJD"] -= bran_disc.mjd

print(xray_data)

erange = np.array(["0.2-10" for x in xray_data["MJD"]])

xray_data = xray_data.sort_values(["MJD"], ascending=True)
xray_data["instrument"][xray_data["instrument"] == "XRT"] = "Swift-XRT"
xray_data["instrument"][xray_data["instrument"] == "XMM"] = "XMM-Newton"
erange[xray_data["instrument"] == "XMM"] = "0.3-10"
xray_data["flux"] *= 1.e12
xray_data["flux_err"] *= 1.e12
xray_data = xray_data.assign(erange=erange)
xray_data = xray_data.round(2)

print(xray_data)


fields = ["MJD", "flux", "flux_err", "erange", "instrument"]

col_headings = [
    r"$\Delta_{t}$",
    r"Energy Flux [10$^{-12}$ erg cm$^{-2}$ s$^{-1}$]",
    r"Flux Err [10$^{-12}$ erg cm$^{-2}$ s$^{-1}$]",
    "Energy Range [keV]",
    "Instrument"
]


# if __name__ == '__main__':
# Basic document
doc = Document('basic')

# fill_document(doc, photometry_data.iloc[:, : 4], col_headings)
fill_document(doc, xray_data[fields], col_headings)


# Add stuff to the document
with doc.create(Section('A second section')):
    doc.append('Some text.')

tex = doc.dumps()  # The document as string in LaTeX syntax
print(tex)

         #MJD       #date         MJD     UL  counts  counts_err  \
0   58620.209         NaN   37.369206  False  0.0622      0.0061   
1   58624.083         NaN   41.243206  False  0.0185      0.0098   
2   58627.207         NaN   44.367206  False  0.0287      0.0067   
3   58630.324         NaN   47.484206  False  0.0502      0.0089   
4   58633.586         NaN   50.746206  False  0.0349      0.0050   
5   58636.198         NaN   53.358206  False  0.0190      0.0038   
6   58639.852         NaN   57.012206  False  0.0027      0.0015   
7   58642.438         NaN   59.598206  False  0.0113      0.0033   
8   58645.426         NaN   62.586206  False  0.0056      0.0022   
9   58648.796         NaN   65.956206  False  0.0071      0.0034   
10  58653.777         NaN   70.937206   True     NaN         NaN   
11  58675.555         NaN   92.715206   True     NaN         NaN   
12  58680.328         NaN   97.488206   True     NaN         NaN   
13  58693.602         NaN  110.762206   True    

<ipython-input-5-5a25a4f1dc7c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xray_data["instrument"][xray_data["instrument"] == "XRT"] = "Swift-XRT"
<ipython-input-5-5a25a4f1dc7c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xray_data["instrument"][xray_data["instrument"] == "XMM"] = "XMM-Newton"


make table with equiparition fitting results
and
table with all the radio data

In [6]:

dd = {}
# epsion_B = 0.001
# epsion_e = 0.1
# eta = epsion_B/epsion_e / (6/11)

# Eeq_corr = (11/17)*eta**(-6/17) + (6/17) *eta**(11/17)
# Req_corr = eta**(1/17)
# print (Req_corr, Eeq_corr)

# new: just the radio data
print ('\n')

data_rec = radio_rec
data_rec = data_rec[data_rec['inst']!='eMERLIN']

mjd0 = bran_disc.mjd
mjds = np.array([58625, 58653, 58703, 58761, 58819])

for i, mjd in enumerate(mjds): 
    
    it = abs(data_rec['mjd']-mjd)<14
    ii = it * (data_rec['eflux_mJy']>0)

    epoch_data  = data_rec[ii]
    for j in np.argsort(epoch_data ['nu_GHz']):
        print ('{0:6.0f}&{1:7.2f}&{2:5.3f}&{3:5.3f}&{4:9}\\\\'.format(epoch_data[j]['mjd']-mjd0, epoch_data[j]['nu_GHz'], epoch_data[j]['flux_mJy'], epoch_data[j]['eflux_mJy'], epoch_data[j]['inst']))
    print ('''\hline''')









    42&   8.49&0.290&0.032&VLA      \\
    42&   9.51&0.408&0.035&VLA      \\
    42&  10.49&0.440&0.036&VLA      \\
    42&  11.51&0.412&0.037&VLA      \\
    41&  15.50&0.464&0.045&AMI      \\
\hline
    70&   1.40&0.104&0.018&MeerKAT  \\
    70&   3.50&0.103&0.028&VLA      \\
    70&   4.49&0.319&0.040&VLA      \\
    70&   5.51&0.324&0.033&VLA      \\
    70&   6.49&0.443&0.037&VLA      \\
    70&   7.51&0.558&0.041&VLA      \\
    70&   8.49&0.680&0.045&VLA      \\
    70&   9.51&0.730&0.047&VLA      \\
    70&  10.49&0.756&0.048&VLA      \\
    70&  11.51&0.771&0.051&VLA      \\
    71&  15.50&0.730&0.076&AMI      \\
\hline
   111&   1.40&0.111&0.017&MeerKAT  \\
   120&   2.24&0.249&0.065&VLA      \\
   120&   2.76&0.345&0.053&VLA      \\
   120&   3.18&0.255&0.049&VLA      \\
   120&   3.69&0.419&0.043&VLA      \\
   120&   4.74&0.698&0.046&VLA      \\
   120&   5.76&0.829&0.053&VLA      \\
   120&   6.69&0.987&0.058&VLA      \\
   120&   7.71&1.117&0.063&VLA      \\
   120&  

In [7]:
# read fit results
for eps_str in ['Eqp','nonEqp']:
    dd[eps_str] = astropy.io.ascii.read(os.path.join(data_dir, 'at2019dsg_mcmc_time_{0}.dat'.format(eps_str)), format='fixed_width')

# print table 1:
for i, d1 in enumerate(dd['Eqp']):
    d01=dd['nonEqp'][i]
    ss = '{0:4} & ${1:5.2f}\pm{2:0.2f}$ & '.format((int(d1['time'])),10**d1['F_p'], np.log(10)*d1['eF_p']*10**d1['F_p'] )
    ss += '${0:5.1f}\pm{1:0.1f}$ \\\\ '.format(10**d1['nu_p']/1e9, np.log(10)*d1['enu_p']*10**d1['nu_p']/1e9 )
    print (ss)
print ('\n\n')

  42 & $ 0.41\pm0.04$ & $ 14.7\pm1.2$ \\ 
  70 & $ 0.71\pm0.04$ & $ 11.9\pm0.5$ \\ 
 120 & $ 1.19\pm0.04$ & $  9.3\pm0.3$ \\ 
 178 & $ 1.25\pm0.04$ & $  5.4\pm0.1$ \\ 





In [8]:

# print table 2:
for geo in ['cone', 'sphere']:
    print ('')
    for i, d1 in enumerate(dd['Eqp']):
        d01=dd['nonEqp'][i]
        ss = '&{0:4}& '.format((int(d1['time'])))
        ss += '${0:0.2f}({1:0.2f})$ & '.format(d1['R_'+geo], d1['eR_'+geo])
        ss += '${0:0.1f}({1:0.1f})$ &'.format(d1['E_'+geo], d1['eE_'+geo])
        ss += '${0:0.2f}({1:0.2f})$ & '.format(d1['B_'+geo], d1['eB_'+geo])
        ss += '${0:0.1f}({1:0.1f})$ &'.format(d1['n_electr_'+geo], d1['en_electr_'+geo])
        ss += '${0:0.2f}({1:0.2f})$ & '.format(d01['R_'+geo], d01['eR_'+geo])
        ss += '${0:0.1f}({1:0.1f})$ & '.format(d01['E_'+geo], d01['eE_'+geo])
        ss += '${0:0.2f}({1:0.2f})$ & '.format(d01['B_'+geo], d01['eB_'+geo])
        ss += '${0:0.1f}({1:0.1f})$ \\\\'.format(d01['n_electr_'+geo], d01['en_electr_'+geo])

        print (ss)



&  42& $16.28(0.03)$ & $48.0(0.1)$ &$-0.25(0.05)$ & $3.5(0.1)$ &$16.18(0.02)$ & $49.4(0.1)$ & $-0.67(0.05)$ & $4.4(0.1)$ \\
&  70& $16.48(0.02)$ & $48.3(0.1)$ &$-0.37(0.04)$ & $3.3(0.1)$ &$16.37(0.02)$ & $49.7(0.1)$ & $-0.79(0.04)$ & $4.1(0.1)$ \\
& 120& $16.68(0.02)$ & $48.7(0.1)$ &$-0.48(0.04)$ & $3.1(0.1)$ &$16.58(0.02)$ & $50.1(0.1)$ & $-0.91(0.04)$ & $3.9(0.1)$ \\
& 178& $16.93(0.02)$ & $49.0(0.1)$ &$-0.72(0.04)$ & $2.6(0.1)$ &$16.82(0.02)$ & $50.3(0.1)$ & $-1.16(0.04)$ & $3.4(0.1)$ \\

&  42& $15.91(0.03)$ & $47.4(0.1)$ &$0.01(0.05)$ & $4.0(0.1)$ &$15.81(0.02)$ & $48.8(0.1)$ & $-0.41(0.04)$ & $4.9(0.1)$ \\
&  70& $16.11(0.02)$ & $47.8(0.1)$ &$-0.10(0.04)$ & $3.8(0.1)$ &$16.00(0.02)$ & $49.2(0.1)$ & $-0.52(0.04)$ & $4.7(0.1)$ \\
& 120& $16.31(0.02)$ & $48.2(0.1)$ &$-0.22(0.04)$ & $3.6(0.1)$ &$16.21(0.01)$ & $49.5(0.1)$ & $-0.65(0.04)$ & $4.4(0.1)$ \\
& 178& $16.56(0.02)$ & $48.4(0.1)$ &$-0.46(0.04)$ & $3.1(0.1)$ &$16.45(0.02)$ & $49.8(0.1)$ & $-0.89(0.04)$ & $3.9(0.1)$ \\
